In [91]:
import json
from datetime import datetime, time

from pymongo.mongo_client import MongoClient
import bson

from datetime import datetime
import os

* A partir du fichier trips_.json, remplacez les espaces des clés par un underscore, puis mettez en base en changeant les chaîne de caractères qui concerne des grandeurs numériques par le type integer ou float et les champs de date ($date) doit être sous format datetime.datetime. Ainsi les champs préfixé '$number...' suivent  l'exemple ci-dessous(cherchez du côté de la librairie bson, il existe une fonction utilitaire, il faut aussi réduire la profondeur):
* From the file trips_.json, replace the spaces in the keys with underscores. Then insert the data into the database while converting string values that represent numerical quantities into either integer or float types. Fields containing dates ($date) must be converted to the datetime.datetime format. Also, fields prefixed with $number... should follow the example below (look into the bson library — there is a utility function for this). The depth of the document should also be reduced.

```json
{'tripduration': {'$numberInt': 379}}
```

en :

```json
{'tripduration': 379}
```

In [92]:

def clean_document(doc):
    if isinstance(doc, dict):
        # Handle special MongoDB extended JSON values
        if "$numberInt" in doc:
            return int(doc["$numberInt"]) #If the document is like: {"$numberInt": "42"}, this will return 42 as a Python int.
        elif "$numberDouble" in doc:
            return float(doc["$numberDouble"])
        elif "$numberLong" in doc:
            return int(doc["$numberLong"])
        elif "$date" in doc:
            date_val = doc["$date"]
            if isinstance(date_val, dict) and "$numberLong" in date_val:
                return datetime.fromtimestamp(int(date_val["$numberLong"]) / 1000.0) #Dates are stored in JSON as a Unix timestamp in milliseconds Python's datetime.fromtimestamp() expects seconds, so we divide by 1000.
            elif isinstance(date_val, str): #If the date is an ISO-formatted string
                return datetime.fromisoformat(date_val.replace("Z", "+00:00")) #.replace("Z", "+00:00") makes it compatible with datetime.fromisoformat
                                                                                #Converts the string to a native datetime object

        # Clean key names and recursively clean values
        cleaned = {}
        for key, value in doc.items():
            clean_key = key.replace(" ", "_")
            cleaned[clean_key] = clean_document(value) #handling normal dictionaries (not just special MongoDB fields)
        return cleaned

    elif isinstance(doc, list): #f doc is a list
        return [clean_document(item) for item in doc] #this goes through each item in the list and applies clean_document() recursively.
    
    else:
        return doc #If the input doc is: a string ("hello"), an integer (123), a float (3.14), None, or anything that isn’t a dict or list…, 
                   #Then it just returns it as-is.


In [93]:
import json

cleaned_documents = [] #Creates an empty list that will store all the cleaned documents.

with open("trips_.json", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip() #removes leading/trailing whitespace and newline characters
        if not line:
            continue
        raw_doc = json.loads(line) #converts the JSON string into a Python dictionary 
        cleaned = clean_document(raw_doc) #applies the cleaning function I defined earlier
        cleaned_documents.append(cleaned) #Adds the cleaned version of the document to the cleaned_documents list

# Show one example
print(cleaned_documents[0])



{'tripduration': 307, 'start_station_id': 3118, 'start_station_name': 'McGuinness Blvd & Eagle St', 'end_station_id': 3119, 'end_station_name': 'Vernon Blvd & 50 Ave', 'bikeid': 23477, 'usertype': 'Subscriber', 'birth_year': 1987, 'gender': 1, 'start_station_location': {'type': 'Point', 'coordinates': [-73.95284, 40.73555]}, 'end_station_location': {'type': 'Point', 'coordinates': [-73.95411749, 40.74232744]}, 'start_time': datetime.datetime(2016, 1, 1, 8, 4, 31), 'stop_time': datetime.datetime(2016, 1, 1, 8, 9, 38)}


Après avoir mis en base et uniquement, assurez-vous qu'il n'y a pas de doublons, si il y en a, écrivez une fonction python permettant de les retirer (sans à avoir à faire de téléchargement) 'delete dublicates'

In [94]:
# Connect to local MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["bikedata"]
collection = db["trips"]


# Clear existing documents to avoid duplicate stacking
collection.delete_many({})
print("Cleared existing documents.")

# Insert cleaned documents
collection.insert_many(cleaned_documents)
print(f"Inserted {len(cleaned_documents)} documents.")


Cleared existing documents.
Inserted 10019 documents.


In [95]:
from bson.json_util import dumps #converts a Python dict (or MongoDB document) into a JSON-formatted string, 
                                #including support for special MongoDB types like ObjectId, datetime, etc.

def remove_duplicates(collection):
    seen = set()
    duplicates = []

    for doc in collection.find(): #Loop Through Every Document in the Collection
        doc_copy = doc.copy() #Prepare the Document for Comparison
        doc_copy.pop("_id", None) #Even if two documents are otherwise identical, their _ids will differ, so we remove _id before comparison.
        doc_str = dumps(doc_copy, sort_keys=True) #Converts the document into a string using bson.json_util.dumps()
                                                #sort_keys=True ensures a consistent order for fields 

        if doc_str in seen: #If this stringified document has been seen before → it’s a duplicate → store its _id in duplicates
            duplicates.append(doc["_id"])
        else:
            seen.add(doc_str) #If not → add it to seen

    if duplicates:
        collection.delete_many({"_id": {"$in": duplicates}}) #Uses $in to match any document with one of those _ids
        print(f"Removed {len(duplicates)} duplicates.")
    else:
        print("No duplicates found.")

remove_duplicates(collection)


Removed 19 duplicates.


In [96]:
output_path = os.path.join(os.getcwd(), "cleaned_trips.json")

with open(output_path, "w", encoding="utf-8") as f:
    for doc in cleaned_documents:
        json.dump(doc, f, default=str)  # default=str for datetime
        f.write("\n")

print("Saved cleaned_trips.json to:")
print(output_path)

Saved cleaned_trips.json to:
/Users/pernebayarailym/Documents/Portfolio_Projects_AP/Simplon_DE_Projects/Python_Projects/MongoDB_practice/cleaned_trips.json


De plus, des hypothèses d'incohérences ont été émises par les parties prenantes, il s'agit d'écrire pour chacune de ces hypothèses une fonction python qui va détecter les documents incriminés et les flagger avec un champs supplémentaire pour rectification/enquête ultérieure :

* Un vélo loué deux fois, mais la deuxième période de location démarre avant le rendu de la première période de location, ce qui n'est pas normal, ou tout autre chevanchement de période.
* Un utilisateur trop jeune (le service est réservé au + de 13 ans)
* La date de naissance n'est pas renseignée
* Des locations trop courtes (1 secondes)
* Des temps de location incohérents par rapport au start_time et au end_time


Faites un rapport détaillé des anomalies trouvées (nombres d'occurences)

* In addition, stakeholders have raised hypotheses about potential data inconsistencies. You are required to write a Python function for each of these hypotheses to detect the problematic documents and flag them with an additional field for future correction/investigation:

- A bike is rented twice, but the second rental period starts before the first one has ended, which is abnormal, or any other overlapping rental periods.
- A user who is too young (the service is restricted to those over 13 years old).
- The birth date is missing.
- Very short rentals (e.g., 1 second long).
- Inconsistent rental durations compared to the recorded start_time and end_time.

Create a detailed report of the anomalies found (number of occurrences).

In [97]:
def detect_overlapping_bike_rentals(collection): #I developed a Python function to detect overlapping bike rentals in the MongoDB dataset.
    from collections import defaultdict #to import defaultdict to automatically create lists for new keys 

    bike_trips = defaultdict(list) #dictionary to group trips by bike ID, e.g., {23477: [trip1, trip2, ...]}
    overlap_count=0 #to count the number of overlapping trips

    #group trips by bike_id
    for trip in collection.find({"start_time":{"$exists": True}, "stop_time": {"$exists": True}}): #sorted each group by start_time to analyze them chronologically.
        bike_trips[trip["bikeid"]].append(trip)

    #check overlap per bike
    for trips in bike_trips.values():
        sorted_trips =sorted(trips, key=lambda x: x["start_time"]) #Then, for each bike, I checked if a new trip started before the previous one ended which would indicate a scheduling conflict or data error.

        for earlier, later in zip(sorted_trips, sorted_trips[1:]):
            if later["start_time"] < earlier["stop_time"]:
                collection.update_one({"_id": later["_id"]}, {"$set": {"anomaly": "overlapping_rental"}}) #If an overlap was found, I flagged the trip by adding an "anomaly": "overlapping_rental" field in the database. 
                overlap_count += 1

    return {"overlapping_rental": overlap_count}

result = detect_overlapping_bike_rentals(collection)
print(result) #Finally, I returned the total number of overlaps detected for reporting purposes.

{'overlapping_rental': 2}


In [98]:
from datetime import datetime # we import datetime to get the current year (to calculate user's ages)

def detect_underage_users(collection): # define a function that takes a MongoDB collection as input
    current_year = datetime.now().year # Get the current year (eg 2025) so we can calculate the user;s age

#count how many users have a birth year that means they are younger than 13
    underage_count = collection.count_documents({
        "birth_year": { "$exists": True, #make sure birth_year field exists
                       "$ne": "", #And it's not an empty string
                       "$gt": current_year - 13} #and birth year is greater than (current year - 13) = too young
    })

    # tag those name underage users with an "anomaly" field in the database
    collection.update_many(
        {"birth_year": {"$exists": True, "$ne": "", "$gt":current_year-13}}, #same filter as above to update
        {"$set": {"anomaly": "underage_user"}} #add or update the anomaly field in their document
    )

    return {"underage_user": underage_count} #return a report with the number of underage users detected

result = detect_underage_users(collection) #call the function and store the result
print(result)

{'underage_user': 2}


Puis, après avoir fait un nettoyage et écarté les locations suspectes, répondez aux questions métier qui suivent :

* Changez le gender 0 en gender 1

* Quels sont les 5 trajets (start station → end station) les plus fréquents pour les utilisateurs de genre féminin ? (gender = 1)

* Quel est le nombre total de trajets par type d’utilisateur (Subscriber vs Customer) pour le premier jour de l'année ?

* Quelle est la durée moyenne des trajets par station de départ pour les trajets commençant entre 7h et 9h ?

* Quel est le top 3 des stations avec la plus forte fréquentation de prise de location, entre 6h et 8h ?

* Quelle est la durée médiane des trajets pour les + de 65 ans ?

* Quelle est la répartition des trajets (nombre de trajets) par tranche horaire de 2 heures (faire visualisation, 0h-2h, 2h-4h etc..) ?

* Quel est le temps moyen passé en trajet pour chaque genre, filtré sur les trajets de plus de 10 minutes ?

* Combien de trajets ont démarré pour chaque station pendant les heures de pointe (ex. 7h-9h et 17h-19h) ?

* Quel est l'âge le plus courant pour les locations de 18h à 20h ?